In [25]:
from catboost import CatBoostClassifier, Pool
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
train = pd.read_csv('data/raw/train.csv')
test = pd.read_csv('data/raw/sample_submission.csv')

In [27]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
# Получаем последнюю дату в train
last_date = train['Date'].max()

In [28]:
# Создаем маску для декабря 2023
december_mask = (train['Date'].dt.year == 2023) & (train['Date'].dt.month == 12)

# Получаем все пары Company_ID и Product_ID, которые были в декабре 2023
december_pairs = train[december_mask][['Company_ID', 'Product_ID']].drop_duplicates()


In [29]:
# Разделяем sample_submission ID на составляющие
test[['Company_ID', 'Product_ID', 'Month']] = test['Id'].str.split('_', expand=True)
test[['Year', 'Month']] = test['Month'].str.split('-', expand=True)

test['Company_ID'] = test['Company_ID'].astype(int)
test['Product_ID'] = test['Product_ID'].astype(int)
test['Month'] = test['Month'].astype(int)

In [30]:
test

,Id,Target,Company_ID,Product_ID,Month,Year
0,0_1_2024-01,0,0,1,1,2024
1,0_2_2024-01,0,0,2,1,2024
2,0_3_2024-01,0,0,3,1,2024
3,0_5_2024-01,0,0,5,1,2024
4,0_6_2024-01,0,0,6,1,2024
...,...,...,...,...,...,...
113851,3_14664_2024-03,0,3,14664,3,2024
113852,3_14665_2024-03,0,3,14665,3,2024
113853,3_14666_2024-03,0,3,14666,3,2024
113854,3_14667_2024-03,0,3,14667,3,2024


In [31]:
# Создаем уникальные пары Company_ID и Product_ID из sample_submission
submission_pairs = test[['Company_ID', 'Product_ID']].drop_duplicates()

In [32]:

# Находим пары, которых не было в декабре 2023
inactive_pairs = submission_pairs.merge(december_pairs,
                                        on=['Company_ID', 'Product_ID'],
                                        how='left',
                                        indicator=True)
inactive_pairs = inactive_pairs[inactive_pairs['_merge'] == 'left_only'].drop('_merge', axis=1)


In [33]:
# Находим активные пары (которые были в декабре 2023)
active_pairs = submission_pairs.merge(december_pairs,
                                      on=['Company_ID', 'Product_ID'],
                                      how='inner')

print("Количество пар без покупок в декабре 2023:", len(inactive_pairs))
print("Количество пар с покупками в декабре 2023:", len(active_pairs))

Количество пар без покупок в декабре 2023: 22482
Количество пар с покупками в декабре 2023: 15470


In [34]:

# Сохраняем результаты в CSV файлы
inactive_pairs.to_csv('pairs_without_december.csv', index=False)
active_pairs.to_csv('pairs_with_december.csv', index=False)

In [35]:
# Дополнительно можем посмотреть статистику по месяцам
train['Year_Month'] = train['Date'].dt.to_period('M')
monthly_pairs = train.groupby('Year_Month')[['Company_ID', 'Product_ID']].nunique()
monthly_pairs

,Company_ID,Product_ID
Year_Month,,
2019-01,4,6384
2019-02,4,6364
2019-03,4,6501
2019-04,4,6418
2019-05,4,6396
2019-06,4,6709
2019-07,4,6776
2019-08,4,6454
2019-09,4,6599
